In [1]:
from Hubbard.equalizer import *
import numpy as np
from Hubbard.io import *

N = 20
L = 4
Ly = 2
d = 3
shape = 'triangular'
w = 'x'
eqt = 'uvt'
R0 = np.array([3, 3, 7.2])
G = HubbardEqualizer(N,
                     R0=R0,
                     lattice=np.array([L, Ly], dtype=int),
                     band=1,
                     dim=d,
                     avg=0.5,
                     ghost=True,
                     sparse=True,
                     shape=shape,
                     waist=w,
                     equalize=False,
                     lattice_symmetry=False,
                     verbosity=0)
ghost = True
mask = np.ones(G.Nsite, dtype=bool)
err = ValueError('Ghost sites not implemented for this lattice.')


def xy_boundaries(G, N):
    x_bdry = np.concatenate((np.arange(N), np.arange(-N, 0)))
    y_bdry = np.concatenate(
        (np.arange(0, G.Nsite, N), np.arange(N-1, G.Nsite, N)))
    return x_bdry, y_bdry


if ghost:
    if G.lattice_dim == 1:
        mask[[0, -1]] = False
    elif G.lattice_dim == 2:
        if G.lattice_shape == 'square' \
                or G.lattice_shape == 'triangular' and not G.ls:
            Nx, Ny = G.size
            if G.lattice_shape == 'square':
                x_bdry, y_bdry = xy_boundaries(G, Ny)
            elif G.lattice_shape == 'triangular':
                y_bdry, x_bdry = xy_boundaries(G, Nx)
            bdry = [x_bdry, y_bdry]
            mask_axis = np.nonzero(G.size > 2)[0]
            if mask_axis.size != 0:
                masked_idx = np.concatenate([bdry[i] for i in mask_axis])
                mask[masked_idx] = False
        else:
            raise err
    else:
        raise err
    mask_idx = np.where(~mask)[0]
    G.masked_links = squeeze_idx(G.links, mask_idx)
else:
    G.masked_links = G.links
G.mask = mask


lattice: lattice shape is triangular; lattice constants set to: (1520, 1690)
Triangular lattice size adjust to: [4 2]
Equalize: scale_factor is not a number. Set to None.
